In [3]:
from Bio.PDB import PDBParser, NeighborSearch, is_aa
from Bio.SeqUtils import seq1
import pandas as pd
import os
import re

# Load the PDB file
pdb_file = "7phr.pdb"  # Replace with the actual path to your file

# Extract the PDB ID from the file name
pdb_id = os.path.splitext(os.path.basename(pdb_file))[0]

parser = PDBParser(QUIET=True)
structure = parser.get_structure(pdb_id, pdb_file)

# Define a function to parse chain descriptions from the PDB file
def parse_chain_descriptions(pdb_file):
    chain_descriptions = {}
    current_description = ""
    with open(pdb_file, 'r') as f:
        for line in f:
            if line.startswith('COMPND'):
                if 'CHAIN:' in line:
                    match = re.match(r'^COMPND\s+\d+\s+CHAIN:\s+([\w, ]+);', line)
                    if match:
                        chains = match.group(1).replace(" ", "").split(',')
                        for chain_id in chains:
                            chain_descriptions[chain_id] = current_description.strip()
                        current_description = ""
                elif 'MOLECULE:' in line:
                    current_description = line.split(':')[-1].strip()
                else:
                    current_description += ' ' + line.split(':')[-1].strip()
    return chain_descriptions

# Parse chain descriptions
chain_name_mapping = parse_chain_descriptions(pdb_file)

# Extract all atoms in the structure
all_atoms = [atom for atom in structure.get_atoms() if is_aa(atom.get_parent())]

# Build the NeighborSearch object
ns = NeighborSearch(all_atoms)

# Collect all cysteine residues in the structure
cys_residues = []
for model in structure:
    for chain in model:
        for residue in chain:
            if residue.get_resname() == 'CYS':
                cys_residues.append(residue)

# Define a function to get interacting amino acids with their distances
def get_interactions(chain1, chain2, distance_threshold=4.0):
    interactions = []
    for residue1 in chain1:
        if not is_aa(residue1):
            continue
        for residue2 in chain2:
            if not is_aa(residue2):
                continue
            for atom1 in residue1:
                for atom2 in residue2:
                    distance = atom1 - atom2
                    if distance < distance_threshold:
                        interactions.append((residue1, residue2, distance))
                        break
                if (residue1, residue2, distance) in interactions:
                    break
    return interactions

# Function to get Cys-Cys interactions
def get_cys_interactions(cys_residues, disulfide_threshold=2.5, max_cys_distance=4.0):
    disulfide_interactions = []
    non_disulfide_interactions = []
    for i, residue1 in enumerate(cys_residues):
        for residue2 in cys_residues[i + 1:]:
            if 'SG' in residue1 and 'SG' in residue2:
                distance = residue1['SG'] - residue2['SG']
                print(f"Checking Cys-Cys interaction: {residue1} - {residue2}, Distance: {distance}")
                if distance < disulfide_threshold:
                    disulfide_interactions.append((residue1, residue2, distance))
                    print(f"Recorded disulfide Cys-Cys interaction: {residue1} - {residue2}, Distance: {distance}")
                elif distance < max_cys_distance:
                    non_disulfide_interactions.append((residue1, residue2, distance))
                    print(f"Recorded non-disulfide Cys-Cys interaction: {residue1} - {residue2}, Distance: {distance}")
    return disulfide_interactions, non_disulfide_interactions

# Extract chain interactions
chain_interactions = {}
chains = list(structure[0])
for i, chain1 in enumerate(chains):
    for j, chain2 in enumerate(chains):
        if i >= j:
            continue
        interactions = get_interactions(chain1, chain2)
        if interactions:
            chain_interactions[(chain1.id, chain2.id)] = interactions

# Get Cys-Cys interactions
disulfide_interactions, non_disulfide_interactions = get_cys_interactions(cys_residues)

# Prepare the interaction data in a readable format
interaction_data = []
for (chain1_id, chain2_id), interactions in chain_interactions.items():
    for res1, res2, distance in interactions:
        interaction_data.append({
            "Chain 1": chain1_id,
            "Chain 1 Description": chain_name_mapping.get(chain1_id, "Unknown"),
            "Residue 1": f"{res1.get_resname()} {res1.id[1]}",
            "Chain 2": chain2_id,
            "Chain 2 Description": chain_name_mapping.get(chain2_id, "Unknown"),
            "Residue 2": f"{res2.get_resname()} {res2.id[1]}",
            "Distance": distance
        })

# Prepare the Cys-Cys interaction data in a readable format
disulfide_interaction_data = []
for res1, res2, distance in disulfide_interactions:
    disulfide_interaction_data.append({
        "Chain 1": res1.get_parent().id,
        "Chain 1 Description": chain_name_mapping.get(res1.get_parent().id, "Unknown"),
        "Cys 1": f"{res1.get_resname()} {res1.id[1]}",
        "Chain 2": res2.get_parent().id,
        "Chain 2 Description": chain_name_mapping.get(res2.get_parent().id, "Unknown"),
        "Cys 2": f"{res2.get_resname()} {res2.id[1]}",
        "Distance": distance
    })
    print(f"Appended verified disulfide Cys-Cys interaction: {res1} - {res2}")

non_disulfide_interaction_data = []
for res1, res2, distance in non_disulfide_interactions:
    non_disulfide_interaction_data.append({
        "Chain 1": res1.get_parent().id,
        "Chain 1 Description": chain_name_mapping.get(res1.get_parent().id, "Unknown"),
        "Cys 1": f"{res1.get_resname()} {res1.id[1]}",
        "Chain 2": res2.get_parent().id,
        "Chain 2 Description": chain_name_mapping.get(res2.get_parent().id, "Unknown"),
        "Cys 2": f"{res2.get_resname()} {res2.id[1]}",
        "Distance": distance
    })
    print(f"Appended verified non-disulfide Cys-Cys interaction: {res1} - {res2}")

# Extract chain sequences and identify missing residues
chain_sequences = []
for chain in chains:
    sequence = "".join([seq1(residue.get_resname()) for residue in chain if is_aa(residue)])
    present_residues = [residue.id[1] for residue in chain if is_aa(residue)]
    all_residues = list(range(min(present_residues), max(present_residues) + 1))
    missing_residues = list(set(all_residues) - set(present_residues))
    chain_sequences.append({
        "Chain": chain.id,
        "Chain Description": chain_name_mapping.get(chain.id, "Unknown"),
        "Sequence": sequence,
        "Missing Residues": missing_residues
    })

# Create DataFrames for the interaction details
interaction_df = pd.DataFrame(interaction_data)
disulfide_interaction_df = pd.DataFrame(disulfide_interaction_data)
non_disulfide_interaction_df = pd.DataFrame(non_disulfide_interaction_data)
chain_sequence_df = pd.DataFrame(chain_sequences)

# Prepare a summary of interactions
summary_data = []
for (chain1_id, chain2_id), interactions in chain_interactions.items():
    summary_data.append({
        "Chain 1": chain1_id,
        "Chain 1 Description": chain_name_mapping.get(chain1_id, "Unknown"),
        "Chain 2": chain2_id,
        "Chain 2 Description": chain_name_mapping.get(chain2_id, "Unknown"),
        "Number of Interactions": len(interactions)
    })

# Create a DataFrame for the summary
summary_df = pd.DataFrame(summary_data)

# Ensure interaction data is not empty
if disulfide_interaction_df.empty:
    print("Warning: No disulfide Cys-Cys interactions detected.")
else:
    print("Verified disulfide Cys-Cys interactions detected and will be saved to Excel.")

if non_disulfide_interaction_df.empty:
    print("Warning: No non-disulfide Cys-Cys interactions detected.")
else:
    print("Verified non-disulfide Cys-Cys interactions detected and will be saved to Excel.")

# Save all DataFrames to an Excel file with different sheets
output_file = f"Interacting_Amino_Acids_and_Chains_{pdb_id}.xlsx"

with pd.ExcelWriter(output_file) as writer:
    interaction_df.to_excel(writer, sheet_name="Interactions", index=False)
    summary_df.to_excel(writer, sheet_name="Summary", index=False)
    disulfide_interaction_df.to_excel(writer, sheet_name="Disulfide Cys-Cys", index=False)
    non_disulfide_interaction_df.to_excel(writer, sheet_name="Non-Disulfide Cys-Cys", index=False)
    chain_sequence_df.to_excel(writer, sheet_name="Chain Sequences", index=False)

print(f"Results have been saved to '{output_file}'")


Checking Cys-Cys interaction: <Residue CYS het=  resseq=24 icode= > - <Residue CYS het=  resseq=90 icode= >, Distance: 2.0136656761169434
Recorded disulfide Cys-Cys interaction: <Residue CYS het=  resseq=24 icode= > - <Residue CYS het=  resseq=90 icode= >, Distance: 2.0136656761169434
Checking Cys-Cys interaction: <Residue CYS het=  resseq=24 icode= > - <Residue CYS het=  resseq=133 icode= >, Distance: 43.82508087158203
Checking Cys-Cys interaction: <Residue CYS het=  resseq=24 icode= > - <Residue CYS het=  resseq=183 icode= >, Distance: 45.11671829223633
Checking Cys-Cys interaction: <Residue CYS het=  resseq=24 icode= > - <Residue CYS het=  resseq=205 icode= >, Distance: 55.664276123046875
Checking Cys-Cys interaction: <Residue CYS het=  resseq=24 icode= > - <Residue CYS het=  resseq=24 icode= >, Distance: 25.96916961669922
Checking Cys-Cys interaction: <Residue CYS het=  resseq=24 icode= > - <Residue CYS het=  resseq=92 icode= >, Distance: 24.327733993530273
Checking Cys-Cys interac